In [1]:
using JuMP, Gurobi, LinearAlgebra, Random, Statistics, MAT

In [2]:
J = 4
M = 2

TP = [12, 10, 8, 4]
st = [2, 3];

theta = 10000;

In [3]:
v0 = Model(Gurobi.Optimizer)

@variable(v0, x[0:J,0:J,1:M], Bin)#variavel de ordem
@variable(v0, y[1:J,1:M], Bin)#variavel que indica qual maquina fez a ordem i
@variable(v0, E[0:J]≥0, Int)#variavel de tempo de enfesto
@variable(v0, time>=0)#variavel para funcao objetivo

@constraint(v0 , E[0]==0)# restricao 1
@constraint(v0 , [i=0:J,m=1:M], x[i,i,m]==0)#restricao 2
@constraint(v0, [i=1:J], sum(y[i,m] for m in 1:M) == 1)#restricao 3
@constraint(v0, [i=1:J,m=1:M], sum(x[i,j,m] for j in 0:J)==y[i,m])#restricao 4
@constraint(v0, [j=1:J,m=1:M], sum(x[i,j,m] for i in 0:J)==y[j,m])#restricao 5
@constraint(v0, [m=1:2], sum(x[0,j,m] for j in 1:J)≤ 1)#restricao 6
@constraint(v0, [i=0:J,j=1:J,m=1:M], E[j]≥E[i]+st[m]+TP[j]-(1-x[i,j,m])*theta)#restricao 7
@constraint(v0, [i=1:J], time ≥ E[i])#restricao 8

@objective(v0, Min, time)#funcao objetivo;

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 941647
Academic license 941647 - for non-commercial use only - registered to si___@cos.ufrj.br


time

In [4]:
optimize!(v0)

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-9600K CPU @ 3.70GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads

Academic license 941647 - for non-commercial use only - registered to si___@cos.ufrj.br
Optimize a model with 77 rows, 64 columns and 235 nonzeros
Model fingerprint: 0x1c1a74e1
Variable types: 1 continuous, 63 integer (58 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 24.0000000
Presolve removed 23 rows and 23 columns
Presolve time: 0.00s
Presolved: 54 rows, 41 columns, 176 nonzeros
Variable types: 0 continuous, 41 integer (36 binary)

Root relaxation: objective 0.000000e+00, 35 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  D

In [5]:
objective_value(v0)

22.0

In [6]:
value_x = zeros((5,5,2))
value_x[:,:,:] = value.(v0[:x])[:,:,:]

3-dimensional DenseAxisArray{Float64,3,...} with index sets:
    Dimension 1, 0:4
    Dimension 2, 0:4
    Dimension 3, Base.OneTo(2)
And data, a 5×5×2 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

[:, :, 2] =
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0

In [7]:
value_E = zeros(5)
value_E[:] = value.(v0[:E])[:]

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 0:4
And data, a 5-element Vector{Float64}:
  0.0
 15.0
 22.0
 10.0
 22.0

In [8]:
value_y = zeros((4,2))
value_y[:,:] = value.(v0[:y])[:,:]

4×2 Matrix{Float64}:
 0.0  1.0
 1.0  0.0
 1.0  0.0
 0.0  1.0

In [9]:
Dicionario = Dict(
"x" => value_x,
"y" => value_y,
"E" => value_E        
)

Dict{String, Array{Float64}} with 3 entries:
  "x" => [0.0 0.0 … 1.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 …
  "E" => [0.0, 15.0, 22.0, 10.0, 22.0]
  "y" => [0.0 1.0; 1.0 0.0; 1.0 0.0; 0.0 1.0]

In [10]:
matwrite("v0.mat", Dicionario)